In [13]:
#from __future__ import print_function
# Import MNIST data
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
import tensorflow as tf

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize values between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

print("Training samples:", x_train.shape, y_train.shape)
print("Test samples:", x_test.shape, y_test.shape)

# ----------------------------------
# Define batch_x as the full training dataset
# Define batch_y as the full testing dataset
# ----------------------------------
batch_x = (x_train, y_train)   # Training data (features + labels)
batch_y = (x_test, y_test)     # Testing data (features + labels)

# Check the shapes
print("Batch X (Training) shapes:", batch_x[0].shape, batch_x[1].shape)
print("Batch Y (Testing) shapes:", batch_y[0].shape, batch_y[1].shape)





Training samples: (60000, 28, 28) (60000,)
Test samples: (10000, 28, 28) (10000,)
Batch X (Training) shapes: (60000, 28, 28) (60000,)
Batch Y (Testing) shapes: (10000, 28, 28) (10000,)


In [24]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize values between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# One-hot encode labels (10 classes for digits 0–9)
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

print("Training samples:", x_train.shape, y_train.shape)
print("Test samples:", x_test.shape, y_test.shape)

# ----------------------------------
# Define batch_x as the full training dataset
# Define batch_y as the full testing dataset
# ----------------------------------
batch_x = (x_train, y_train)   # Training data (features + one-hot labels)
batch_y = (x_test, y_test)     # Testing data (features + one-hot labels)

# Check the shapes
print("Batch X (Training) shapes:", batch_x[0].shape, batch_x[1].shape)
print("Batch Y (Testing) shapes:", batch_y[0].shape, batch_y[1].shape)


Training samples: (60000, 28, 28) (60000, 10)
Test samples: (10000, 28, 28) (10000, 10)
Batch X (Training) shapes: (60000, 28, 28) (60000, 10)
Batch Y (Testing) shapes: (10000, 28, 28) (10000, 10)


In [33]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# ---------------------------
# Load and preprocess MNIST
# ---------------------------
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Flatten images: (28,28) → (784,)
x_train = x_train.reshape(-1, 784).astype("float32") / 255.0
x_test = x_test.reshape(-1, 784).astype("float32") / 255.0

# One-hot encode labels
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

print("Training samples:", x_train.shape, y_train.shape)
print("Test samples:", x_test.shape, y_test.shape)

# ---------------------------
# Training Parameters
# ---------------------------
learning_rate = 0.001
num_steps = 500
batch_size = 128
display_step = 100

# Network Parameters
n_hidden_1 = 256   # 1st layer number of neurons
n_hidden_2 = 256   # 2nd layer number of neurons
num_input  = 784   # MNIST data input (28*28 pixels)
num_classes = 10   # MNIST total classes (0–9 digits)

# tf Graph input
x = tf.placeholder("float", [None, num_input])
y = tf.placeholder("float", [None, num_classes])

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Construct model
logits = neural_net(x)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables
init = tf.global_variables_initializer()

# ---------------------------
# Training
# ---------------------------
with tf.Session() as sess:
    sess.run(init)

    for step in range(1, num_steps+1):
        # Get a batch manually
        idx = np.random.randint(0, x_train.shape[0], batch_size)
        batch_x, batch_y = x_train[idx], y_train[idx]

        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={x: batch_x, y: batch_y})

        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy],
                                 feed_dict={x: batch_x, y: batch_y})
            print("Step " + str(step) +
                  ", Minibatch Loss= {:.4f}".format(loss) +
                  ", Training Accuracy= {:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:",
          sess.run(accuracy, feed_dict={x: x_test, y: y_test}))


Training samples: (60000, 784) (60000, 10)
Test samples: (10000, 784) (10000, 10)
Step 1, Minibatch Loss= 3214.6204, Training Accuracy= 0.078
Step 100, Minibatch Loss= 270.7203, Training Accuracy= 0.727
Step 200, Minibatch Loss= 270.2548, Training Accuracy= 0.797
Step 300, Minibatch Loss= 131.1882, Training Accuracy= 0.859
Step 400, Minibatch Loss= 145.7529, Training Accuracy= 0.875
Step 500, Minibatch Loss= 94.8467, Training Accuracy= 0.867
Optimization Finished!
Testing Accuracy: 0.8576
